# Source reconstruction with beamformers in MNE-Python

`
Authors: Britta Westner <britta.wstnr@gmail.com>
         Marijn van Vliet <w.m.vanvliet@gmail.com>
         Alexandre Gramfort <alexandre.gramfort@telecom-paristech.fr>
`

This notebook gives a quick introduction into source reconstruction with beamformers, featuring the _Linearly Constrained Minimum Variance beamformer_ (LCMV) and the _Dynamic Imaging of Coherent Sources_ (DICS).

It will also highlight some of the latest beamforming-related implementations in MNE-Python.



## What is source reconstruction?

<div class="alert alert-info">
    <b>EXPLANATION</b>:
     <ul>
    <li>Source reconstruction for MEG data <b>estimates the source activity</b> that produced the given sensor data. </li>
     <li>For this, a <b> forward model</b> and an <b>inverse model</b> are needed.</li>
         <li> The <b> foward model</b> describes how to go from <b> source to sensor space </b>. </li>
         <li> The <b> inverse model </b> estimates the <b> source activity </b> - using the forward model.</li>
    </ul>
</div>

<img src="sensor_to_source.png" width="700"> 


## What is beamforming?

<div class="alert alert-info">
    <b>EXPLANATION</b>:
     <ul>
         <li>Beamformers are one type of <b> inverse models</b>. </li>
         <li> It uses a so-called <b> spatial filter </b> and estimates source activity <b> independently</b> on every source point.</li>
         <li> It gives back a source space <b> activity pattern </b> and is often used on <b> volumetric </b> source spaces.</li>
    </ul>
</div>

## Beamforming in MNE-Python

<div class="alert alert-success">
    <b>OVERVIEW</b>:
     <ul>
         <li>The beamformer module of MNE-Python got an almost complete makeover.</li>
         <li>Started with Google Summer of Code 2017, (almost) finished with the MNE-Python Coding Sprint 2019. </li>
         <li>Many new functions, new API, new parameters, new plotting, new tests, ...</li>
    </ul>
</div>

# ... let's look at some of this fresh new stuff!

## Loading the data
First, we'll load the data. We will use the ``sample`` dataset (visual and auditory stimulation) and only look at the trials with auditory input.

In [1]:
# for interactive figures:
%matplotlib notebook  

# imports
import mne
from mne.datasets import sample

# paths to data
data_path = sample.data_path()
raw_fname = data_path + '/MEG/sample/sample_audvis_raw.fif'
event_fname = data_path + '/MEG/sample/sample_audvis_raw-eve.fif'
fwd_fname = data_path + '/MEG/sample/sample_audvis-meg-vol-7-fwd.fif'
subjects_dir = data_path + '/subjects'

## Loading and preprocessing the data

In [3]:
event_id, tmin, tmax = [1, 2], -0.2, 0.5

# Setup for reading the raw data
raw = mne.io.read_raw_fif(raw_fname, preload=True)
events = mne.read_events(event_fname)

# Set up pick list: EEG + MEG 
picks = mne.pick_types(raw.info, meg='mag', eeg=False, stim=True, eog=True,
                       exclude='bads')

# Pick the channels of interest
raw.pick_channels([raw.ch_names[pick] for pick in picks])
# Re-normalize our empty-room projectors, so they are fine after subselection
raw.info.normalize_proj()

# Epoching the data
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, 
                    baseline=(None, 0), preload=True, proj=True,
                    reject=dict(mag=4e-12, eog=150e-6))

# Compute the average
evoked = epochs.average()

## Look at our sensor space data

In [4]:
evoked.plot_joint(ts_args=dict(time_unit='s'),
                  topomap_args=dict(time_unit='s'));

## Load the forward model 

To save computation time, we will load the pre-computed forward model for our subject from disk. Note that this is not a _surface_ forward model, but a _volumetric_ forward model - with source points covering the whole volume of the brain.

In [5]:
forward = mne.read_forward_solution(fwd_fname)
forward = mne.convert_forward_solution(forward, surf_ori=True)

## Compute the data covariance matrix

The computation of a beamformer spatial filter needs two ingredients: the forward model and a data covariance matrix.

In [6]:
data_cov = mne.compute_covariance(epochs, tmin=0.01, tmax=0.15, rank=None)

## Compute and apply the beamformer

Now we have all the ingredients to **compute the spatial filter**! We will compute our beamformer with ``make_lcmv`` and then apply it to the evoked data computed above, by using ``apply_lcmv``.

In [7]:
filters = mne.beamformer.make_lcmv(evoked.info, forward, data_cov, pick_ori='max-power')
stc = mne.beamformer.apply_lcmv(evoked, filters)

## Plot the source reconstruction

Let's plot the output - the figure is **interactive**, so we can explore the estimated activity in source space!

In [8]:
stc.plot(src=forward['src'], subject='sample', subjects_dir=subjects_dir);

/Users/rodin/Projects/scikit-learn/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## What is new?

### Functions and API

<div class="alert alert-success">
    <b>2-step API</b>: <br>
    This makes it easy to use pre-computed filters, e.g., for statistical contrasts.
     <ul>
         <li> <tt> make_lcmv </tt> to compute the spatial filter </li>
         <li> <tt> apply_lcmv </tt> for applying the spatial filter to any data </li>
         <li> <tt> apply_lcmv </tt> exists for evoked data, epochs, raw data, and covariance matrixes (e.g., <tt> apply_lcmv_epochs, apply_lcmv_cov</tt>).</li>
    </ul>
</div>

### Cool!

<div class="alert alert-success">
    <b>Mixing channel types</b>: <br>
    By supplying a <b> noise covariance matrix </b> it is possible to use several sensor types (e.g., MEG + EEG or mags + grads) for beamforming. <br>
    The noise covariance matrix is used to <b> whiten the leadfield and data covariance matrix. </b>
</div>


### Parameters and functionality (selection)

<div class="alert alert-success">
    <b>Supported is</b>: 
    <ul>
        <li> <b> Leadfield normalization </b> using <tt> depth</tt>. </li>
        <li> <b> Weight normalization </b> using <tt> weight_norm</tt>, this also enables estimation of the <b> Neural Activity Index</b>. </li>
        <li> <b> Orientation selection </b>, e.g., normal to surface or maximal power, using <tt> pick_ori</tt>.
</div>
    
    
### Across toolbox stability

<div class="alert alert-success">
    <b>Our output converges with FieldTrip's</b> ... <br>
    ... and we constantly run tests (Continuous Integration) to ensure this.   
</div>
    
<img src="fieldtrip_converge.png" width="1000"> 